In [1]:
using PyCall
push!(pyimport("sys")["path"], joinpath(pwd(), "py-mpc"));

In [6]:
@pyimport pympc.geometry as geometry
@pyimport pympc.control as control
@pyimport pympc.dynamical_systems as dynamical_systems
@pyimport pympc.plot as mpcplot
@pyimport scipy.spatial as spatial
@pyimport pympc.optimization.mpqpsolver as mpqpsolver

In [8]:
using JuMP
using Gurobi

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/Gurobi.ji for module Gurobi.


In [24]:
mass = 1.
l = 1.
g = 10.
N = 4
# A = [0. 1.;
#      g/l 0.]
# B = [0 1/(mass*l^2.)]'
A = [0. 1; 0 0]
B = [0. 1]'
Δt = .1
pysys = dynamical_systems.DTLinearSystem[:from_continuous](A, B, Δt)

# x_max = [pi/6, pi/20/(N*Δt)]
x_max = [1., 1]
x_min = -x_max
# u_max = [mass*g*l*pi/8.]
u_max = [1.]
u_min = -u_max
times = 0:Δt:N*Δt

Q = 10 * eye(2)
R = eye(1)

X_bounds = geometry.Polytope[:from_bounds](reshape(x_min, 2, 1), reshape(x_max, 2, 1))[:assemble]()
U_bounds = geometry.Polytope[:from_bounds](reshape(u_min, 1, 1), reshape(u_max, 1, 1))[:assemble]()
controller = control.MPCController(pysys, N, "two", Q, R, X=X_bounds, U=U_bounds)

qp = controller[:condensed_program]
qp[:remove_linear_terms]()
G = qp[:G]
W = qp[:W]
S = qp[:S]
H = qp[:H]
H_inv = qp[:H_inv]


4×4 Array{Float64,2}:
  0.378559   -0.081126   -0.0492823  -0.0230304
 -0.081126    0.411865   -0.0539786  -0.0254564
 -0.0492823  -0.0539786   0.43599    -0.0303987
 -0.0230304  -0.0254564  -0.0303987   0.461698 

In [25]:
x = x_min + rand(length(x_min)) .* (x_max - x_min)

2-element Array{Float64,1}:
 0.807867
 0.402321

In [65]:
function mpc(qp, x)
    G = qp[:G]
    W = qp[:W]
    S = qp[:S]
    H = qp[:H]
    H_inv = qp[:H_inv]
    
    m = Model(solver=GurobiSolver(OutputFlag=0))
    @variable m z[1:size(G, 2)]
    @objective m Min (z' * H * z)[1]
    @constraint m constrs G * z .<= S * x + W
    status = solve(m, suppress_warnings=true)
    if status == :Optimal
        lambdas = vec(getdual.(constrs))
        active_set = abs.(lambdas) .>= 1e-4

        G_A = G[active_set, :]
        W_A = W[active_set, :]
        S_A = S[active_set, :]
        G_I = G[!active_set, :]
        W_I = W[!active_set, :]
        S_I = S[!active_set, :]
        H_A = inv(G_A * H_inv * G_A')

        lambda_A_offset = -H_A * W_A
        lambda_A_linear = -H_A * S_A

        z_offset = -H_inv * G_A' * lambda_A_offset
        z_linear = -H_inv * G_A' * lambda_A_linear

        u_offset = z_offset - H_inv * qp[:F_u]
        u_linear = z_linear - H_inv * qp[:F_xu]'

        status, u_offset .+ u_linear * x, u_linear
    else
        status, fill(NaN, length(u), 1), fill(NaN, length(u), length(x))
    end
end

mpc (generic function with 1 method)

In [73]:
for i in 1:20
    x = x_min + rand(length(x_min)) .* (x_max - x_min)
    status, u, J = mpc(qp, x)
    if status == :Optimal
        @assert isapprox(u, controller[:feedforward](x)[1], atol=1e-5)
        for i in 1:length(x)
            delta = zeros(x)
            delta[i] += 1e-3
            x2 = x .+ delta
            s2, u2, J2 = mpc(qp, x2)
            @assert isapprox(u2, u + J * delta, atol=1e-5)
        end
    end
end